# Evaluate Vector Representation
- word vector가 아닌 sentence vector로 뉴스기사의 유사도를 비교합니다

1. 'news_sample.txt' 파일을 load한다. 이 때, 개별 뉴스기사의 첫번째 문장만 활용한다. 예를 들어, 1번 뉴스기사에 41개 문장이 있다면, 그 중 첫번째 문장만 선택한다.
2.   word2vec과 fastText 모델을 load한다
3. word vector로 sentence vector를 만든다.
  1. 문장을 단어(또는 토큰)단위로 나눈다.
  2. 단어의 embedding vector를 word2vec과 fastText 모델에서 불러온다.
  3. **문장의 모든 단어의 embedding vector값을 전부 합친다.**
  4. 합쳐준 값을 문장벡터로 간주한다.
4. 이제 각 뉴스기사의 요약의 embedding vector를 활용해서, 개별 뉴스기사가 서로 유사한지 비교한다(cosine similarity 사용)




In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = '/content/drive/MyDrive/PBL_0114'
filename = 'news_sample.txt'

### 전처리

In [3]:
!pip install hanja
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu, mac 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

import re, hanja
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

EMAIL_PATTERN = re.compile(r'''(([a-zA-Z0-9._%+-]+)@([a-zA-Z0-9.-]+)(\.[a-zA-Z]{2,4}))''', re.VERBOSE)
URL_PATTERN = re.compile("(ftp|http|https)?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", re.VERBOSE)
MULTIPLE_SPACES = re.compile(' +', re.UNICODE)

def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        tokenizer = Kkma()
        
    return tokenizer

def tokenize(tokenizer_name, original_sent, pos=False):
    tokenizer = get_tokenizer(tokenizer_name)
    sentence = original_sent.replace('\n', '').strip()
    if pos:
        tokens = tokenizer.pos(sentence)
        tokens = [morph + "/" + tag for morph, tag in tokens]
    else:
      # tokenizer.nouns(sentence) -> 명사만 추출
        tokens = tokenizer.morphs(sentence)
        
    # tokenized_sent = ' '.join(post_process(tokens))
    tokenized_sent = ' '.join(tokens)
    
    return tokenized_sent

def cleansing_other(sentence: str = None) -> str:
    """
    문장을 전처리 (이메일, URL, 공백 등 제거) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub(EMAIL_PATTERN, ' ', sentence)
    sentence = re.sub(URL_PATTERN, ' ', sentence)
    sentence = re.sub(MULTIPLE_SPACES, ' ', sentence)
    sentence = sentence.replace(", )", "")
    
    return sentence

def cleansing_chinese(sentence: str = None) -> str:
    """
    한자를 변환하는 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    # chinese character를 앞뒤로 괄호가 감싸고 있을 경우, 대부분 한글 번역임
    sentence = re.sub("\([\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]+\)", "", sentence)
    # 다른 한자가 있다면 한글로 치환
    if re.search("[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]", sentence) is not None:
        sentence = hanja.translate(sentence, 'substitution')

    return sentence

def cleansing_special(sentence: str = None) -> str:
    """
    특수문자를 전처리를 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence: str = None) -> str:
    """
    숫자를 전처리(delexicalization) 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

def preprocess_sent(sentence: str = None) -> str:
    """
    모든 전처리를 수행 하는 함수
    :param sentence: 전처리 대상 문장
    :return: 전처리 완료된 문장
    """
    sent_clean = sentence
    sent_clean = cleansing_other(sent_clean)
    sent_clean = cleansing_chinese(sent_clean)
    sent_clean = cleansing_special(sent_clean)
    sent_clean = cleansing_numbers(sent_clean)
    sent_clean = re.sub('\s+', ' ', sent_clean)
    sent_clean = tokenize('mecab', sent_clean)

    return sent_clean

     |████████████████████████████████| 122kB 12.7MB/s 
     |████████████████████████████████| 266kB 10.7MB/s 
     |████████████████████████████████| 245kB 23.8MB/s 
  Created wheel for hanja: filename=hanja-0.13.2-cp36-none-any.whl size=128339 sha256=224c17beec83cccb85dc7e14beb67cae2d6e2cba4e81c450a474b1fe9dcd57d7
  Stored in directory: /root/.cache/pip/wheels/dc/86/35/26e2202156c2371a71b443de1bab5c957169ee6f1a9eb77c98
  Created wheel for pyyaml: filename=PyYAML-5.1.2-cp36-cp36m-linux_x86_64.whl size=44105 sha256=c6dfc5fdb9c35593ac747aad4b0645a655fd9797b8c5789a588fc93fca3c3c4e
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built hanja pyyaml
ERROR: datascience 0.10.6 has requirement coverage==3.7.1, but you'll have coverage 5.3.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: coveralls 0.5 has requirement coverage<3.999,

## 1번
1. 'news_sample.txt' 파일을 load한다.
2. 이 때, 개별 뉴스기사의 첫 3개 문장을 활용한다. 예를 들어, 1번 뉴스기사에 41개 문장이 있다면, 그 중 1번부터 3번까지만 선택한다.
  - 이를 위해 `kss` 를 설치해야한다
3. 3개 문장을 전부 하나로 연결한다. 이 때, 띄어쓰기(' ')를 활용한다. 이를 뉴스의 요약이라고 간주한다
4. 각 뉴스기사의 요약을 전처리한다




In [4]:
!pip install kss

In [5]:
import kss

In [12]:
news_summ = []
with open(os.path.join(path, filename), 'r', encoding='utf-8') as f:
  for line in f.readlines():
    sent_list = kss.split_sentences(line)
    news_summ.append(sent_list)

In [13]:
news_summ

[['새로운 희망 공유하고 새 출발하자 기축년(己丑年) 새해다.',
  '새로운 희망을 공유하고 새 출발을 다짐할 때다.',
  '짙푸른 동해바다를 뚫고 이글거리는 광채를 뿜으며 힘차게 솟아오르는 태양을 안는다.',
  '어제와 다른 이 아침의 햇살을 받으며 왜 간절한 소망이 없겠는가.',
  '지난날의 갈등과 혼란 회한을 뒤로하고 새로 일어서야 한다.',
  '올해는 불신과 증오, 암투와 음모, 좌절과 절망 등 어둡고 참담한 것들이 걷히기를 소망한다.',
  '경제위기라고 하지만 우리는 저력을 가지고 있다.',
  '남북관계 새 틀 모색해야 우리의 역사는 위기와 극복의 과정으로 이어져 왔다.',
  '시련과 고난을 극복하는 과정에서 오히려 민족의 역량을 키우고 한 단계 발전을 이룩해 왔던 것이다.',
  '달군 쇠가 단단해지고, 비 온 뒤의 땅이 굳어진다는 것은 마치 우리 민족을 두고 한 듯하다.',
  '이것은 우리의 무서운 생명력이다.',
  '그 피가 지금 우리 몸속에 흐르고 있다.',
  '‘대한민국호’는 글로벌 경제위기라는 미증유의 시련을 떨쳐내고 선진국으로 도약해야 한다.',
  '국민과 정부가 한마음 한뜻으로 경제 살리기에 나선다면 재도약은 못 이룰 꿈도 아니다.',
  '이를 위해서는 첫 단추를 잘 끼워야 한다.',
  '정부가 해야 할 몫은 경제에 자생력과 추진력을 불러일으키는 것이다.',
  '기업과 국민이 의욕적으로 다시 뛸 수 있도록 동기를 부여하고 여건을 만들어 주는 일이다.',
  '이를 바탕으로 변화의 바람을 몰고 올 ‘오바마 코드’를 제대로 읽어야 한다.',
  '새해 한국 외교의 최대 과제는 오바마 시대와의 ‘지혜로운 조화’라고 할 수 있다.',
  '미국 건국 후 흑인으로는 처음으로 대통령에 취임하는 오바마의 정책을 잘 파악해 한국의 국익을 극대화할 수 있는 새로운 한미관계를 정립해 나가야 한다.',
  '새해에는 남북관계도 새 틀이 모색되어야 한다.',
  '지난해에는 당국 간 대화 단절에 이어 금강산과 개성관광 중단, 경의

In [28]:
preprocessed_sum = [preprocess_sent(summ) for summ in news_summ]

TypeError: ignored

## 2번

-   word2vec과 fastText 모델을 load한다


In [15]:
from gensim.models import KeyedVectors

In [16]:
w2v_model = KeyedVectors.load(os.path.join(path, 'word2vec_model'), mmap='r')

In [17]:
ft_model = KeyedVectors.load_word2vec_format(os.path.join(path, 'fastText_model.vec'))

## 3번
- word vector로 sentence vector를 만든다.
  1. 문장을 단어(또는 토큰)단위로 나눈다.
  2. 단어의 embedding vector를 word2vec과 fastText 모델에서 불러온다.
  3. **문장의 모든 단어의 embedding vector값을 전부 합친다.**
  4. 합쳐준 값을 문장벡터로 간주한다. 합쳐진 벡터를 summ_vec라고 한다

In [ ]:
'''

setn_vec_list =
[[a1, ..., a300],
 [b1, ..., b300],
 [c1, ..., c300]]

sent_vec = [a1+b1+c1, ..., a300+b300+c300]

'''

In [23]:
import numpy as np
import pandas as pd

In [24]:
# model.wv.vocab.keys()
# model['word'(변수명)]
def get_sent_vector(model, sentence):
  sent_vec = []
  for word in sentence.split(' '):
    # out of vocabulary = oov (모델안에 해당단어가 있는지 확인해야됨)
    if word in model.wv.vocab.keys():
      sent_vec.append(model[word])
    return sent_vec

In [ ]:
# summ_vec를 만든다
# np.summ(sent_vec, axis=0)
results = []
for idx, summ in enumerate(preprocessed_sum):
  sent_vec = get_sent_vector(ft_model, summ)
  summ_vec = np.sum(sent_vec, axis=0)
  results.append((idx, summ_vec))

In [ ]:
df = pd.DataFrame(data=results, columns=['news_id','summ_vectors'])

## 4번

- 이제 각 뉴스기사의 요약의 embedding vector를 활용해서, 개별 뉴스기사가 서로 유사한지 비교한다(cosine similarity 사용)
- https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
result_df = pd.DataFrame(cosine_similarity([x.tolist() for x in df['summ_vectors']]))